In [14]:
import re
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib
from tensorflow.keras.models import load_model
from eunjeon import Mecab
import numpy as np
import pickle

In [15]:
# 토크나이저 로드 함수
def load_tokenizer(filepath):
    with open(filepath, 'rb') as f:
        tokenizer = pickle.load(f)
    return tokenizer

In [16]:
from konlpy.tag import Mecab

In [23]:
data = pd.read_csv('./data/tokenized_data_ALL.csv', encoding_errors='ignore', low_memory=False)
data['new_label']=None
data['tokenized_comment']=None
loaded_model = load_model('./model/final_model_ALL.h5')
tokenizer = Tokenizer()
mecab = Mecab('C:/mecab/mecab-ko-dic')

In [24]:
# 예측 과정에서의 토크나이저 로드
tokenizer = load_tokenizer('./model/tokenizer_ALL.pkl')
stopwords = load_tokenizer('./model/stopwords_ALL.pkl')


In [25]:
def sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence)
    new_sentence = [word for word in new_sentence if word not in stopwords]
    
    if not new_sentence:
        print("토큰이 없어 중립 리뷰입니다.")
        return -1
    
    # 시퀀스 길이 설정
    max_len = 80
    
    # 시퀀스를 인코딩하고 패딩
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen=max_len)
    
    # 모델에 입력하고 예측 수행
    score = loaded_model.predict(pad_new)[0]
    
    score = float(score)  # 배열에서 스칼라 값으로 변환
    
    if score > 0.5:
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
        return 1
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
        return 0


In [26]:
data['tokenized_comments'] = None

In [27]:
tokenizer.index_word

{1: 'OOV',
 2: '좋',
 3: '맛있',
 4: '었',
 5: '어요',
 6: 'ㄴ',
 7: '습니다',
 8: '았',
 9: '어',
 10: '먹',
 11: '어서',
 12: '맛',
 13: '시',
 14: '는데',
 15: '음식',
 16: '분위기',
 17: 'ㅂ니다',
 18: '있',
 19: '친절',
 20: '네요',
 21: '아요',
 22: '많',
 23: 'ㄹ',
 24: '같',
 25: '아',
 26: '주',
 27: '없',
 28: '예약',
 29: '것',
 30: '괜찮',
 31: '오',
 32: '분',
 33: '직원',
 34: '아서',
 35: '방문',
 36: '만족',
 37: '서비스',
 38: '지만',
 39: '않',
 40: '나오',
 41: '기',
 42: '아쉽',
 43: '더',
 44: '자리',
 45: '곳',
 46: '면',
 47: '라',
 48: '파스타',
 49: '피자',
 50: '니',
 51: '보',
 52: '거',
 53: '나',
 54: '수',
 55: '넓',
 56: '때',
 57: '메뉴',
 58: 'ㅁ',
 59: '식사',
 60: '사람',
 61: '주문',
 62: '갈',
 63: '되',
 64: '시간',
 65: '가격',
 66: '편하',
 67: '공연',
 68: 'ㅎㅎ',
 69: '음',
 70: '강남',
 71: '그렇',
 72: '딸기',
 73: '싶',
 74: '네이버',
 75: '신선로',
 76: '집',
 77: '감사',
 78: '다음',
 79: '굿',
 80: 'ㄴ데',
 81: '크',
 82: '맛나',
 83: '최고',
 84: '겠',
 85: '비싸',
 86: '알',
 87: '구요',
 88: '받',
 89: '구',
 90: '늦',
 91: '아니',
 92: '매장',
 93: '생각',
 94: '친구',
 95: '이용',


In [28]:
for index, row in data.iterrows():
    sp = sentiment_predict(row['Review'])
    if sp == 1:
        data.at[index, 'new_label'] = '1'
    elif sp == -1:
        data.at[index, 'new_label'] = '-1'
    else:
        data.at[index, 'new_label'] = '0'

    # mecab.pos 함수를 호출하기 전에 body가 str 타입인지 확인
    if isinstance(row['Review'], str):
        tokenized_comments = []
        pos_tags = mecab.pos(row['Review'])
        for token, pos in pos_tags:
            # 형태소가 동사(VV) 또는 형용사(VA)인 경우에만 tokenized_comments에 추가
            if pos in ['Noun', 'Verb', 'Adjective','Exclamation']:
                tokenized_comments.append(token)
        data.at[index, 'tokenized_comments'] = tokenized_comments

1/1 [==============================] - 0s 424ms/step
100.00% 확률로 부정 리뷰입니다.
1/1 [==============================] - 0s 35ms/step
97.17% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 36ms/step


C:\Users\USER\AppData\Local\Temp\ipykernel_3904\2931568357.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(score)  # 배열에서 스칼라 값으로 변환


99.96% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 55ms/step
98.75% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 47ms/step
99.28% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 36ms/step
99.89% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 39ms/step
99.97% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 32ms/step
99.45% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 30ms/step
99.14% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 33ms/step
96.48% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 39ms/step
99.89% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 31ms/step
99.97% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 31ms/step
96.25% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 33ms/step
99.98% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 57ms/step
99.60% 확률로 부정 리뷰입니다.
1/1 [==============================] - 0s 33ms/step
99.54% 확률로 긍정 리뷰입니다.
1/1 [=========================

In [29]:
# 토크나이저의 단어 인덱스 확인
print(tokenizer.word_index)

# 모델의 임베딩 층의 가중치 가져오기
embedding_layer = loaded_model.layers[0]  # 임베딩 층의 인덱스는 모델의 첫 번째 층일 가능성이 높습니다.
embedding_weights = embedding_layer.get_weights()[0]

# 임베딩 가중치의 크기 출력
print(embedding_weights.shape)  # (vocab_size, embedding_dim)

{'OOV': 1, '좋': 2, '맛있': 3, '었': 4, '어요': 5, 'ㄴ': 6, '습니다': 7, '았': 8, '어': 9, '먹': 10, '어서': 11, '맛': 12, '시': 13, '는데': 14, '음식': 15, '분위기': 16, 'ㅂ니다': 17, '있': 18, '친절': 19, '네요': 20, '아요': 21, '많': 22, 'ㄹ': 23, '같': 24, '아': 25, '주': 26, '없': 27, '예약': 28, '것': 29, '괜찮': 30, '오': 31, '분': 32, '직원': 33, '아서': 34, '방문': 35, '만족': 36, '서비스': 37, '지만': 38, '않': 39, '나오': 40, '기': 41, '아쉽': 42, '더': 43, '자리': 44, '곳': 45, '면': 46, '라': 47, '파스타': 48, '피자': 49, '니': 50, '보': 51, '거': 52, '나': 53, '수': 54, '넓': 55, '때': 56, '메뉴': 57, 'ㅁ': 58, '식사': 59, '사람': 60, '주문': 61, '갈': 62, '되': 63, '시간': 64, '가격': 65, '편하': 66, '공연': 67, 'ㅎㅎ': 68, '음': 69, '강남': 70, '그렇': 71, '딸기': 72, '싶': 73, '네이버': 74, '신선로': 75, '집': 76, '감사': 77, '다음': 78, '굿': 79, 'ㄴ데': 80, '크': 81, '맛나': 82, '최고': 83, '겠': 84, '비싸': 85, '알': 86, '구요': 87, '받': 88, '구': 89, '늦': 90, '아니': 91, '매장': 92, '생각': 93, '친구': 94, '이용': 95, '시끄럽': 96, '즐겁': 97, '추천': 98, '모임': 99, '테이블': 100, '였': 101, '빠르': 102, '점심': 103, '디': 104,

In [30]:
data.to_csv('./data/data_ALL.csv', encoding='utf-8-sig')

In [31]:
# 'tokenized' 열에서 빈 리스트인 경우 NaN으로 처리
data['tokenized'] = data['tokenized'].apply(lambda x: np.nan if len(x) == 2 else x)

# NaN 값 제거
data.dropna(subset=['tokenized'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [32]:
import pandas as pd

In [33]:
data

,Unnamed: 0,Rating,Review,nouns,verbs,adjectives,exclamations,tokenized,new_label,tokenized_comment,tokenized_comments
0,0,0,손님 놓치기 싫어서 지하층으로 손님을 받을거면 동일한 서비스를 제공하세요 그동안 ...,"['손님', '지하층', '손님', '거', '동일', '서비스', '제공', '앞...","['놓치', '싫', '받', '이', '가', '갈', '이', '하', '달라'...","['싫', '없']","['기', '어서', '을', '면', 'ㄴ', '세요', '었', '지만', 'ㅂ...","['놓치', '싫', '받', '이', '가', '갈', '이', '하', '달라'...",0,None,[]
1,1,1,김치찌개랑 계란말이 쨩,"['김치찌개', '계란', '말']",[],[],[],"['김치찌개', '계란', '말']",1,None,[]
2,2,1,맛있어요,[],['맛있'],['맛있'],['어요'],"['맛있', '맛있', '어요']",1,None,[]
3,3,1,바빠요,[],['바쁘'],['바쁘'],['아요'],"['바쁘', '바쁘', '아요']",1,None,[]
4,4,1,삼굡살이 너무맛나게먹엇쳐욤,[],"['맛나', '치']",['맛나'],"['게', '어요']","['맛나', '치', '맛나', '게', '어요']",1,None,[]
...,...,...,...,...,...,...,...,...,...,...,...
10134,10206,1,주차 시간 무료,"['주차', '시간', '무료']",[],[],[],"['주차', '시간', '무료']",1,None,[]
10135,10207,1,케익이 맛있는 아티제,"['케익', '티']","['맛있', '아']",['맛있'],"['는', '아']","['맛있', '아', '맛있', '는', '아', '케익', '티']",1,None,[]
10136,10208,1,주차가돼요,['주차'],['되'],[],['어요'],"['되', '어요', '주차']",1,None,[]
10137,10209,1,깔끔하고 저녁엔 회사 사람들이 빠져나가서 조용하고 좋네요,"['저녁', '회사', '사람']","['빠져나가', '좋']",['좋'],"['고', '서', '고', '네요']","['빠져나가', '좋', '좋', '고', '서', '고', '네요', '저녁', ...",1,None,[]


In [34]:
data['Rating'].dtype

dtype('int64')

In [35]:
data['new_label'] = data['new_label'].astype('int32')

In [36]:
data['new_label'].dtype

dtype('int32')

In [37]:
same_count = (data['Rating'] == data['new_label']).sum()
different_count = len(data) - same_count

# 결과 출력
print(f"같은 것의 개수: {same_count}")
print(f"다른 것의 개수: {different_count}")

같은 것의 개수: 9797
다른 것의 개수: 342


In [38]:
# TN, TP, FN, FP 계산
TN = ((data['Rating'] == 0) & (data['new_label'] == 0)).sum()
TP = ((data['Rating'] == 1) & (data['new_label'] == 1)).sum()
FN = ((data['Rating'] == 1) & (data['new_label'] == 0)).sum()
FP = ((data['Rating'] == 0) & (data['new_label'] == 1)).sum()

# 결과 출력
print(f"TN: {TN}")
print(f"TP: {TP}")
print(f"FN: {FN}")
print(f"FP: {FP}")

TN: 354
TP: 9443
FN: 190
FP: 152


In [39]:
# 전체 샘플 수
total_samples = len(data)

# 정확하게 예측된 샘플 수
correct_predictions = TP + TN

# 정확도 계산
accuracy = correct_predictions / total_samples

# 결과 출력
print(f"정확도: {accuracy}")

정확도: 0.966268862806983
